In [1]:
import os
import sys
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pyspark.sql.types as sqlt
import pyspark.sql.functions as sqlf
from pyspark import SparkConf
try:
    sc.stop()
except:
    pass
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

conf = (SparkConf()
.setAppName('test') 
.setMaster("yarn")
.set('spark.yarn.queue', "production")
.set("spark.executor.cores", "5")
.set("spark.driver.cores", "5")
.set('spark.dynamicAllocation.enabled', 'false')
.set("spark.driver.memory", "2g")
.set("spark.executor.memory", "2g"))
        
conf.setMaster('yarn')
conf.setAppName('spark-test')
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)
spark = sqlContext
sql = spark.sql

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/12 23:10:40 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/08/12 23:10:40 WARN util.Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/08/12 23:10:40 WARN util.Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/08/12 23:10:40 WARN util.Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
23/08/12 23:10:44 WARN lineage.LineageWriter: Lineage directory /var/log/spark/lineage doesn't exist or is not writable. Lineage for this application will be disabled.


## Для заданий 2-7 сравните скорость выполнения с использованием broadcast и cache и без них.


Задание 1. Загрузите файлы из hdfs:///user/data_loader/FIFA в Spark DataFrame. Запишите таблицы в parquet hive table в свою БД, убедитесь, что заголовки и типы данных корректны.https://www.kaggle.com/datasets/abhijitdahatonde/fifa-world-cup-all-dataset


In [2]:
# CREATE DATABASE ivanov; # hive

In [3]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, DateType
employeesSchema1 = StructType([
    StructField("id", IntegerType()),
    StructField("Year", IntegerType()),
    StructField("Stage", StringType()),
    StructField("Home_Team", StringType()),
    StructField("Away_Team", StringType()),
    ])

In [4]:
employeesSchema2 = StructType([
    StructField("id", IntegerType()),
    StructField("Tournament", StringType()),
    StructField("Date", DateType()),
    StructField("Home_Team", StringType()),
    StructField("Away_Team", StringType()),
    StructField("Home_Goals", IntegerType()),
    StructField("Away_Goals", IntegerType()),
    StructField("Home_Stadium_or_Not", IntegerType()),
   ])

In [5]:
employeesSchema3 = StructType([
    StructField("id", IntegerType()),
    StructField("Year", IntegerType()),
    StructField("Stage", StringType()),
    StructField("Home_Team", StringType()),
    StructField("Home_Goals", IntegerType()),
    StructField("Away_Goals", IntegerType()),
    StructField("Away_Team", StringType()),
    StructField("Host_Team_or_Not", IntegerType()),
    ])

In [6]:
employeesSchema4 = StructType([
    StructField("Year", IntegerType()),
    StructField("Host_Country", StringType()),
    StructField("Winner", StringType()),
    StructField("Runners_Up", StringType()),
    StructField("Third", StringType()),
    StructField("Fourth", StringType()),
    StructField("Goals_Scored", IntegerType()),
    StructField("Matches_Played", IntegerType()),
    StructField("Qualified_Teams", IntegerType())
])

In [7]:
df1 = spark.read.csv('hdfs:///user/std03/2022_world_cup_matches1.csv', header = True, schema = employeesSchema1)

23/08/12 23:10:49 WARN lineage.LineageWriter: Lineage directory /var/log/spark/lineage doesn't exist or is not writable. Lineage for this application will be disabled.


In [8]:
df2 = spark.read.csv('hdfs:///user/std03/international_matches1.csv', header = True, schema = employeesSchema2)

In [9]:
df3 = spark.read.csv('hdfs:///user/std03/world_cup_matches1.csv', header = True, schema = employeesSchema3)

In [10]:
df4 = spark.read.csv('hdfs:///user/std03/world_cups1.csv', header = True, schema = employeesSchema4)

In [11]:
df_1 = df1.write.format('parquet').mode('overwrite').saveAsTable('ivanov.2022_world_cup_matches1_p')

In [12]:
df_2 = df2.write.format('parquet').mode('overwrite').saveAsTable('ivanov.international_matches1_p')

23/08/12 23:10:57 WARN hive.HiveExternalCatalog: Could not persist `ivanov`.`international_matches1_p` in a Hive compatible way. Persisting it into Hive metastore in Spark SQL specific format.
org.apache.hadoop.hive.ql.metadata.HiveException: java.lang.UnsupportedOperationException: Parquet does not support date. See HIVE-6384
	at org.apache.hadoop.hive.ql.metadata.Hive.createTable(Hive.java:868)
	at org.apache.hadoop.hive.ql.metadata.Hive.createTable(Hive.java:873)
	at org.apache.spark.sql.hive.client.HiveClientImpl$$anonfun$createTable$1.apply$mcV$sp(HiveClientImpl.scala:489)
	at org.apache.spark.sql.hive.client.HiveClientImpl$$anonfun$createTable$1.apply(HiveClientImpl.scala:487)
	at org.apache.spark.sql.hive.client.HiveClientImpl$$anonfun$createTable$1.apply(HiveClientImpl.scala:487)
	at org.apache.spark.sql.hive.client.HiveClientImpl$$anonfun$withHiveState$1.apply(HiveClientImpl.scala:283)
	at org.apache.spark.sql.hive.client.HiveClientImpl.liftedTree1$1(HiveClientImpl.scala:221)


In [13]:
df_3 = df3.write.format('parquet').mode('overwrite').saveAsTable('ivanov.world_cup_matches1_p')

In [14]:
df_4 = df4.write.format('parquet').mode('overwrite').saveAsTable('ivanov.world_cups1_p')

In [15]:
df2.schema

StructType(List(StructField(id,IntegerType,true),StructField(Tournament,StringType,true),StructField(Date,DateType,true),StructField(Home_Team,StringType,true),StructField(Away_Team,StringType,true),StructField(Home_Goals,IntegerType,true),StructField(Away_Goals,IntegerType,true),StructField(Home_Stadium_or_Not,IntegerType,true)))

In [16]:
df1 = spark.table('ivanov.2022_world_cup_matches1_p')
df1.show()

+---+----+-----------+-------------+-------------+
| id|Year|      Stage|    Home_Team|    Away_Team|
+---+----+-----------+-------------+-------------+
|  1|2022|Group stage|        Qatar|      Ecuador|
|  2|2022|Group stage|      Senegal|  Netherlands|
|  3|2022|Group stage|      England|         Iran|
|  4|2022|Group stage|United States|        Wales|
|  5|2022|Group stage|       France|    Australia|
|  6|2022|Group stage|      Denmark|      Tunisia|
|  7|2022|Group stage|       Mexico|       Poland|
|  8|2022|Group stage|    Argentina| Saudi Arabia|
|  9|2022|Group stage|      Belgium|       Canada|
| 10|2022|Group stage|        Spain|   Costa Rica|
| 11|2022|Group stage|      Germany|        Japan|
| 12|2022|Group stage|      Morocco|      Croatia|
| 13|2022|Group stage|  Switzerland|     Cameroon|
| 14|2022|Group stage|      Uruguay|  South Korea|
| 15|2022|Group stage|     Portugal|        Ghana|
| 16|2022|Group stage|       Brazil|       Serbia|
| 17|2022|Group stage|         

In [17]:
df2 = spark.table('ivanov.international_matches1_p')
df2.show()

+---+----------+----------+----------------+---------+----------+----------+-------------------+
| id|Tournament|      Date|       Home_Team|Away_Team|Home_Goals|Away_Goals|Home_Stadium_or_Not|
+---+----------+----------+----------------+---------+----------+----------+-------------------+
|  1|  Friendly|1872-11-30|        Scotland|  England|         0|         0|                  1|
|  2|  Friendly|1873-03-08|         England| Scotland|         4|         2|                  1|
|  3|  Friendly|1874-03-07|        Scotland|  England|         2|         1|                  1|
|  4|  Friendly|1875-03-06|         England| Scotland|         2|         2|                  1|
|  5|  Friendly|1876-03-04|        Scotland|  England|         3|         0|                  1|
|  6|  Friendly|1876-03-25|        Scotland|    Wales|         4|         0|                  1|
|  7|  Friendly|1877-03-03|         England| Scotland|         1|         3|                  1|
|  8|  Friendly|1877-03-05|   

In [18]:
spark.sql(''' 
            select DISTINCT Stage
            from ivanov.world_cup_matches1_p
                        
            ''').show(truncate=False)

+------------------+
|Stage             |
+------------------+
|Final             |
|First group stage |
|Group stage       |
|Third place       |
|First round       |
|Round of 16       |
|Semi-finals       |
|Final round       |
|Second group stage|
|Quarter-finals    |
+------------------+



In [19]:
df3 = spark.table('ivanov.world_cup_matches1_p')
df3.show()

+---+----+-----------+-------------+----------+----------+-------------+----------------+
| id|Year|      Stage|    Home_Team|Home_Goals|Away_Goals|    Away_Team|Host_Team_or_Not|
+---+----+-----------+-------------+----------+----------+-------------+----------------+
|  1|1930|Group stage|       France|         4|         1|       Mexico|               0|
|  2|1930|Group stage|United States|         3|         0|      Belgium|               0|
|  3|1930|Group stage|   Yugoslavia|         2|         1|       Brazil|               0|
|  4|1930|Group stage|      Romania|         3|         1|         Peru|               0|
|  5|1930|Group stage|    Argentina|         1|         0|       France|               0|
|  6|1930|Group stage|        Chile|         3|         0|       Mexico|               0|
|  7|1930|Group stage|   Yugoslavia|         4|         0|      Bolivia|               0|
|  8|1930|Group stage|United States|         3|         0|     Paraguay|               0|
|  9|1930|

In [20]:
%%time
df4 = spark.table('ivanov.world_cups1_p')
df4.show()

+----+------------+----------+--------------+-----------+--------------+------------+--------------+---------------+
|Year|Host_Country|    Winner|    Runners_Up|      Third|        Fourth|Goals_Scored|Matches_Played|Qualified_Teams|
+----+------------+----------+--------------+-----------+--------------+------------+--------------+---------------+
|1930|     Uruguay|   Uruguay|     Argentina|        USA|    Yugoslavia|          70|            18|             13|
|1934|       Italy|     Italy|Czechoslovakia|    Germany|       Austria|          70|            17|             16|
|1938|      France|     Italy|       Hungary|     Brazil|        Sweden|          84|            18|             15|
|1950|      Brazil|   Uruguay|        Brazil|     Sweden|         Spain|          88|            22|             13|
|1954| Switzerland|Germany FR|       Hungary|    Austria|       Uruguay|         140|            26|             16|
|1958|      Sweden|    Brazil|        Sweden|     France|    Ger

Задание 2. Проведите дескриптивный анализ данных: определите первичные и внешние ключи таблиц, посчитайте количества записей, распределение категориальных признаков, статистики по числовым признакам, заполненность данных, размер таблиц.


In [21]:
# pip install pandas

In [22]:
t1 = df1.toPandas()

In [23]:
t1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         64 non-null     int32 
 1   Year       64 non-null     int32 
 2   Stage      64 non-null     object
 3   Home_Team  64 non-null     object
 4   Away_Team  64 non-null     object
dtypes: int32(2), object(3)
memory usage: 2.1+ KB


In [24]:
t1.describe(include = 'all')

,id,Year,Stage,Home_Team,Away_Team
count,64.000000,64.0,64,64,64
unique,NaN,NaN,6,45,45
top,NaN,NaN,Group stage,Qatar,Cameroon
freq,NaN,NaN,48,3,3
mean,32.500000,2022.0,NaN,NaN,NaN
std,18.618987,0.0,NaN,NaN,NaN
min,1.000000,2022.0,NaN,NaN,NaN
25%,16.750000,2022.0,NaN,NaN,NaN
50%,32.500000,2022.0,NaN,NaN,NaN
75%,48.250000,2022.0,NaN,NaN,NaN


In [25]:
t1.shape

(64, 5)

In [26]:
t2 = df2.toPandas()

In [27]:
t2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17769 entries, 0 to 17768
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   17769 non-null  int32 
 1   Tournament           17769 non-null  object
 2   Date                 17769 non-null  object
 3   Home_Team            17769 non-null  object
 4   Away_Team            17769 non-null  object
 5   Home_Goals           17769 non-null  int32 
 6   Away_Goals           17769 non-null  int32 
 7   Home_Stadium_or_Not  17769 non-null  int32 
dtypes: int32(4), object(4)
memory usage: 833.0+ KB


In [28]:
t2.describe(include = 'all')

,id,Tournament,Date,Home_Team,Away_Team,Home_Goals,Away_Goals,Home_Stadium_or_Not
count,17769.000000,17769,17769,17769,17769,17769.000000,17769.000000,17769.000000
unique,NaN,90,9004,214,218,NaN,NaN,NaN
top,NaN,Friendly,2009-08-12,Mexico,Uruguay,NaN,NaN,NaN
freq,NaN,7871,27,520,521,NaN,NaN,NaN
mean,8885.000000,NaN,NaN,NaN,NaN,1.722382,1.169002,0.804997
std,5129.612802,NaN,NaN,NaN,NaN,1.692834,1.335989,0.396214
min,1.000000,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000
25%,4443.000000,NaN,NaN,NaN,NaN,1.000000,0.000000,1.000000
50%,8885.000000,NaN,NaN,NaN,NaN,1.000000,1.000000,1.000000
75%,13327.000000,NaN,NaN,NaN,NaN,2.000000,2.000000,1.000000


In [29]:
t2.shape

(17769, 8)

In [30]:
t3 = df3.toPandas()

In [31]:
t3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                900 non-null    int32 
 1   Year              900 non-null    int32 
 2   Stage             900 non-null    object
 3   Home_Team         900 non-null    object
 4   Home_Goals        900 non-null    int32 
 5   Away_Goals        900 non-null    int32 
 6   Away_Team         900 non-null    object
 7   Host_Team_or_Not  900 non-null    int32 
dtypes: int32(5), object(3)
memory usage: 38.8+ KB


In [32]:
t3.describe(include = 'all')

,id,Year,Stage,Home_Team,Home_Goals,Away_Goals,Away_Team,Host_Team_or_Not
count,900.000000,900.000000,900,900,900.000000,900.000000,900,900.000000
unique,NaN,NaN,10,77,NaN,NaN,81,NaN
top,NaN,NaN,Group stage,Brazil,NaN,NaN,Mexico,NaN
freq,NaN,NaN,528,79,NaN,NaN,36,NaN
mean,450.500000,1986.915556,NaN,NaN,1.792222,1.038889,NaN,0.131111
std,259.951919,23.150270,NaN,NaN,1.593279,1.081567,NaN,0.337710
min,1.000000,1930.000000,NaN,NaN,0.000000,0.000000,NaN,0.000000
25%,225.750000,1970.000000,NaN,NaN,1.000000,0.000000,NaN,0.000000
50%,450.500000,1990.000000,NaN,NaN,2.000000,1.000000,NaN,0.000000
75%,675.250000,2006.000000,NaN,NaN,3.000000,2.000000,NaN,0.000000


In [33]:
t3.shape

(900, 8)

In [34]:
t4 = df4.toPandas()

In [35]:
t4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Year             21 non-null     int32 
 1   Host_Country     21 non-null     object
 2   Winner           21 non-null     object
 3   Runners_Up       21 non-null     object
 4   Third            21 non-null     object
 5   Fourth           21 non-null     object
 6   Goals_Scored     21 non-null     int32 
 7   Matches_Played   21 non-null     int32 
 8   Qualified_Teams  21 non-null     int32 
dtypes: int32(4), object(5)
memory usage: 1.3+ KB


In [36]:
t4.describe(include = 'all')

,Year,Host_Country,Winner,Runners_Up,Third,Fourth,Goals_Scored,Matches_Played,Qualified_Teams
count,21.000000,21,21,21,21,21,21.000000,21.000000,21.000000
unique,NaN,16,9,11,15,16,NaN,NaN,NaN
top,NaN,Italy,Brazil,Argentina,Germany,Uruguay,NaN,NaN,NaN
freq,NaN,2,5,3,3,3,NaN,NaN,NaN
mean,1976.857143,NaN,NaN,NaN,NaN,NaN,121.333333,42.857143,21.761905
std,26.657618,NaN,NaN,NaN,NaN,NaN,33.943090,17.467930,7.462605
min,1930.000000,NaN,NaN,NaN,NaN,NaN,70.000000,17.000000,13.000000
25%,1958.000000,NaN,NaN,NaN,NaN,NaN,89.000000,32.000000,16.000000
50%,1978.000000,NaN,NaN,NaN,NaN,NaN,126.000000,38.000000,16.000000
75%,1998.000000,NaN,NaN,NaN,NaN,NaN,146.000000,64.000000,32.000000


In [37]:
t4.shape

(21, 9)

Задание 3. Найдите все матчи чемпионата мира, в которых хотя бы одна команда является победителем или финалистом прошлого чемпионата.


In [38]:
%%time
spark.sql('''
            with t1 as(
            select Home_Team as team from ivanov.world_cup_matches1_p 
            where Stage = 'Final'
            and Year = 2018
            union
            select Away_Team from ivanov.world_cup_matches1_p 
            where Stage = 'Final'
            and Year = 2018)
            
            select t2.* from ivanov.2022_world_cup_matches1_p as t2, t1
            where t2.Home_Team = t1.team or t2.Away_Team = t1.team
            
            ''').show(truncate=False)

+---+----+-----------+---------+---------+
|id |Year|Stage      |Home_Team|Away_Team|
+---+----+-----------+---------+---------+
|5  |2022|Group stage|France   |Australia|
|23 |2022|Group stage|France   |Denmark  |
|38 |2022|Group stage|Tunisia  |France   |
|12 |2022|Group stage|Morocco  |Croatia  |
|27 |2022|Group stage|Croatia  |Canada   |
|41 |2022|Group stage|Croatia  |Belgium  |
+---+----+-----------+---------+---------+

CPU times: user 2.22 ms, sys: 400 µs, total: 2.62 ms
Wall time: 1.74 s


In [39]:
df = spark.table('ivanov.world_cup_matches1_p').cache()

In [40]:
%%time
spark.sql('''
            with t1 as(
            select Home_Team as team from ivanov.world_cup_matches1_p 
            where Stage = 'Final'
            and Year = 2018
            union
            select Away_Team from ivanov.world_cup_matches1_p 
            where Stage = 'Final'
            and Year = 2018)
            
            select t2.* from ivanov.2022_world_cup_matches1_p as t2, t1
            where t2.Home_Team = t1.team or t2.Away_Team = t1.team
            
            ''').show(truncate=False)

+---+----+-----------+---------+---------+
|id |Year|Stage      |Home_Team|Away_Team|
+---+----+-----------+---------+---------+
|5  |2022|Group stage|France   |Australia|
|23 |2022|Group stage|France   |Denmark  |
|38 |2022|Group stage|Tunisia  |France   |
|12 |2022|Group stage|Morocco  |Croatia  |
|27 |2022|Group stage|Croatia  |Canada   |
|41 |2022|Group stage|Croatia  |Belgium  |
+---+----+-----------+---------+---------+

CPU times: user 1.82 ms, sys: 1.31 ms, total: 3.13 ms
Wall time: 1.1 s


In [41]:
df.unpersist()

DataFrame[id: int, Year: int, Stage: string, Home_Team: string, Home_Goals: int, Away_Goals: int, Away_Team: string, Host_Team_or_Not: int]

In [42]:
%%time
spark.sql('''
            with t1 as(
            select Home_Team as team from ivanov.world_cup_matches1_p 
            where Stage = 'Final'
            and Year = 2018
            union
            select Away_Team from ivanov.world_cup_matches1_p 
            where Stage = 'Final'
            and Year = 2018)
            
            select /*+ BROADCAST(t1) */ t2.* from ivanov.2022_world_cup_matches1_p as t2, t1
            where t2.Home_Team = t1.team or t2.Away_Team = t1.team
            
            ''').show(truncate=False)

+---+----+-----------+---------+---------+
|id |Year|Stage      |Home_Team|Away_Team|
+---+----+-----------+---------+---------+
|5  |2022|Group stage|France   |Australia|
|12 |2022|Group stage|Morocco  |Croatia  |
|23 |2022|Group stage|France   |Denmark  |
|27 |2022|Group stage|Croatia  |Canada   |
|38 |2022|Group stage|Tunisia  |France   |
|41 |2022|Group stage|Croatia  |Belgium  |
+---+----+-----------+---------+---------+

CPU times: user 1.22 ms, sys: 107 µs, total: 1.33 ms
Wall time: 710 ms


Задание 4. Найдите все матчи чемпионата мира, в которых хозяйствующая страна также участвовала в матче.


In [43]:
%%time
spark.sql('''            
            select * from ivanov.2022_world_cup_matches1_p
            where Home_Team = 'Qatar' or Away_Team = 'Qatar'
            
            ''').show(truncate=False)

+---+----+-----------+---------+-----------+
|id |Year|Stage      |Home_Team|Away_Team  |
+---+----+-----------+---------+-----------+
|1  |2022|Group stage|Qatar    |Ecuador    |
|18 |2022|Group stage|Qatar    |Senegal    |
|36 |2022|Group stage|Qatar    |Netherlands|
+---+----+-----------+---------+-----------+

CPU times: user 1.99 ms, sys: 786 µs, total: 2.77 ms
Wall time: 149 ms


In [44]:
df = spark.table('ivanov.world_cup_matches1_p').cache()

In [45]:
%%time
spark.sql('''            
            select * from ivanov.2022_world_cup_matches1_p
            where Home_Team = 'Qatar' or Away_Team = 'Qatar'
            
            ''').show(truncate=False)

+---+----+-----------+---------+-----------+
|id |Year|Stage      |Home_Team|Away_Team  |
+---+----+-----------+---------+-----------+
|1  |2022|Group stage|Qatar    |Ecuador    |
|18 |2022|Group stage|Qatar    |Senegal    |
|36 |2022|Group stage|Qatar    |Netherlands|
+---+----+-----------+---------+-----------+

CPU times: user 2.1 ms, sys: 396 µs, total: 2.49 ms
Wall time: 145 ms


In [46]:
df.unpersist()

DataFrame[id: int, Year: int, Stage: string, Home_Team: string, Home_Goals: int, Away_Goals: int, Away_Team: string, Host_Team_or_Not: int]

Задание 5. Найдите все матчи чемпионата мира и международные матчи, которые прошли на домашнем стадионе страны, участвовавшей в чемпионате мира. 


In [47]:
%%time
spark.sql(''' 
            with tab as (
            select t1.Stage, t1.Home_Team, t1.Away_Team,
            t2.Tournament, t2.Home_Team as h_t, t2.Away_Team as a_t
            from ivanov.2022_world_cup_matches1_p t1
            join ivanov.international_matches1_p t2 
            on t1.Home_Team = t2.Home_Team)
            
            select Stage, Home_Team, Away_Team from tab
            union all
            select Tournament, h_t, a_t from tab
            
            ''').show(truncate=False)

+-----------+---------+-------------+
|Stage      |Home_Team|Away_Team    |
+-----------+---------+-------------+
|Group stage|England  |Wales        |
|Group stage|England  |United States|
|Group stage|England  |Iran         |
|Group stage|England  |Wales        |
|Group stage|England  |United States|
|Group stage|England  |Iran         |
|Group stage|England  |Wales        |
|Group stage|England  |United States|
|Group stage|England  |Iran         |
|Group stage|England  |Wales        |
|Group stage|England  |United States|
|Group stage|England  |Iran         |
|Group stage|England  |Wales        |
|Group stage|England  |United States|
|Group stage|England  |Iran         |
|Group stage|England  |Wales        |
|Group stage|England  |United States|
|Group stage|England  |Iran         |
|Group stage|England  |Wales        |
|Group stage|England  |United States|
+-----------+---------+-------------+
only showing top 20 rows

CPU times: user 1.98 ms, sys: 1.24 ms, total: 3.22 ms
Wall tim

In [48]:
df1 = spark.table('ivanov.world_cup_matches1_p').cache()
df2 = spark.table('ivanov.international_matches1_p').cache()

In [49]:
%%time
spark.sql(''' 
            with tab as (
            select t1.Stage, t1.Home_Team, t1.Away_Team,
            t2.Tournament, t2.Home_Team as h_t, t2.Away_Team as a_t
            from ivanov.2022_world_cup_matches1_p t1
            join ivanov.international_matches1_p t2 
            on t1.Home_Team = t2.Home_Team)
            
            select Stage, Home_Team, Away_Team from tab
            union all
            select Tournament, h_t, a_t from tab
            
            ''').show(truncate=False)

+-----------+---------+-------------+
|Stage      |Home_Team|Away_Team    |
+-----------+---------+-------------+
|Group stage|England  |Wales        |
|Group stage|England  |United States|
|Group stage|England  |Iran         |
|Group stage|England  |Wales        |
|Group stage|England  |United States|
|Group stage|England  |Iran         |
|Group stage|England  |Wales        |
|Group stage|England  |United States|
|Group stage|England  |Iran         |
|Group stage|England  |Wales        |
|Group stage|England  |United States|
|Group stage|England  |Iran         |
|Group stage|England  |Wales        |
|Group stage|England  |United States|
|Group stage|England  |Iran         |
|Group stage|England  |Wales        |
|Group stage|England  |United States|
|Group stage|England  |Iran         |
|Group stage|England  |Wales        |
|Group stage|England  |United States|
+-----------+---------+-------------+
only showing top 20 rows

CPU times: user 2.6 ms, sys: 209 µs, total: 2.81 ms
Wall time:

In [50]:
df1.unpersist()
df2.unpersist()

DataFrame[id: int, Tournament: string, Date: date, Home_Team: string, Away_Team: string, Home_Goals: int, Away_Goals: int, Home_Stadium_or_Not: int]

In [51]:
%%time
spark.sql(''' 
            with tab as (
            select /*+ BROADCAST(t1) */ t1.Stage, t1.Home_Team, t1.Away_Team,
            t2.Tournament, t2.Home_Team as h_t, t2.Away_Team as a_t
            from ivanov.2022_world_cup_matches1_p t1
            join ivanov.international_matches1_p t2 
            on t1.Home_Team = t2.Home_Team)
            
            select Stage, Home_Team, Away_Team from tab
            union all
            select Tournament, h_t, a_t from tab
            
            ''').show(truncate=False)

+-----------+---------+-------------+
|Stage      |Home_Team|Away_Team    |
+-----------+---------+-------------+
|Group stage|England  |Wales        |
|Group stage|England  |United States|
|Group stage|England  |Iran         |
|Group stage|England  |Wales        |
|Group stage|England  |United States|
|Group stage|England  |Iran         |
|Group stage|England  |Wales        |
|Group stage|England  |United States|
|Group stage|England  |Iran         |
|Group stage|England  |Wales        |
|Group stage|England  |United States|
|Group stage|England  |Iran         |
|Group stage|England  |Wales        |
|Group stage|England  |United States|
|Group stage|England  |Iran         |
|Group stage|England  |Wales        |
|Group stage|England  |United States|
|Group stage|England  |Iran         |
|Group stage|England  |Wales        |
|Group stage|England  |United States|
+-----------+---------+-------------+
only showing top 20 rows

CPU times: user 1.26 ms, sys: 735 µs, total: 1.99 ms
Wall time

Задание 6. Для каждой страны найдите количество выигранных матчей в чемпионатах мира. Используйте оконную функцию для подсчета общего числа побед каждой команды.


In [52]:
%%time
spark.sql(''' 
            with t1 as (
            select DISTINCT Away_Team,
            count(Away_Team) over(partition by Away_Team) cnt_ta
            from ivanov.world_cup_matches1_p
            where Away_Goals > Home_Goals),
            t2 as (
            select DISTINCT Home_Team,
            count(Home_Team) over(partition by Home_Team) cnt_th
            from ivanov.world_cup_matches1_p
            where Home_Goals > Away_Goals),
            t3 as (
            select * from t1
            full join t2
            on t1.Away_Team = t2.Home_Team
            ),
            t4 as (
            select *,
            case when cnt_ta is null then 0 else cnt_ta end c1,
            case when cnt_th is null then 0 else cnt_th end c2
            from t3
            ),
            t5 as (
            select
            case when Away_Team is null then Home_Team
                 when Home_Team is null then Away_Team
                 else Away_Team
            end team,
            (c1+c2) cnt
            from t4
            )
            
            select * from t5
            
            ''').show(truncate=False)

+-------------+---+
|team         |cnt|
+-------------+---+
|Paraguay     |7  |
|Russia       |19 |
|Senegal      |3  |
|Sweden       |19 |
|Turkey       |5  |
|Germany      |67 |
|Ivory Coast  |3  |
|France       |34 |
|Greece       |2  |
|Algeria      |3  |
|Slovakia     |1  |
|Argentina    |43 |
|Wales        |1  |
|Belgium      |20 |
|Ecuador      |4  |
|Ghana        |4  |
|Peru         |5  |
|United States|8  |
|Chile        |11 |
|Croatia      |11 |
+-------------+---+
only showing top 20 rows

CPU times: user 1.46 ms, sys: 1.79 ms, total: 3.25 ms
Wall time: 1.67 s


In [53]:
df1 = spark.table('ivanov.world_cup_matches1_p').cache()

In [54]:
%%time
spark.sql(''' 
            with t1 as (
            select DISTINCT Away_Team,
            count(Away_Team) over(partition by Away_Team) cnt_ta
            from ivanov.world_cup_matches1_p
            where Away_Goals > Home_Goals),
            t2 as (
            select DISTINCT Home_Team,
            count(Home_Team) over(partition by Home_Team) cnt_th
            from ivanov.world_cup_matches1_p
            where Home_Goals > Away_Goals),
            t3 as (
            select * from t1
            full join t2
            on t1.Away_Team = t2.Home_Team
            ),
            t4 as (
            select *,
            case when cnt_ta is null then 0 else cnt_ta end c1,
            case when cnt_th is null then 0 else cnt_th end c2
            from t3
            ),
            t5 as (
            select
            case when Away_Team is null then Home_Team
                 when Home_Team is null then Away_Team
                 else Away_Team
            end team,
            (c1+c2) cnt
            from t4
            )
            
            select * from t5
            
            ''').show(truncate=False)

+-------------+---+
|team         |cnt|
+-------------+---+
|Paraguay     |7  |
|Russia       |19 |
|Senegal      |3  |
|Sweden       |19 |
|Turkey       |5  |
|Germany      |67 |
|Ivory Coast  |3  |
|France       |34 |
|Greece       |2  |
|Algeria      |3  |
|Slovakia     |1  |
|Argentina    |43 |
|Wales        |1  |
|Belgium      |20 |
|Ecuador      |4  |
|Ghana        |4  |
|Peru         |5  |
|United States|8  |
|Chile        |11 |
|Croatia      |11 |
+-------------+---+
only showing top 20 rows

CPU times: user 0 ns, sys: 1.21 ms, total: 1.21 ms
Wall time: 1.01 s


In [55]:
df1.unpersist()

DataFrame[id: int, Year: int, Stage: string, Home_Team: string, Home_Goals: int, Away_Goals: int, Away_Team: string, Host_Team_or_Not: int]

In [56]:
%%time
spark.sql(''' 
            with t1 as (
            select DISTINCT Away_Team,
            count(Away_Team) over(partition by Away_Team) cnt_ta
            from ivanov.world_cup_matches1_p
            where Away_Goals > Home_Goals),
            t2 as (
            select DISTINCT Home_Team,
            count(Home_Team) over(partition by Home_Team) cnt_th
            from ivanov.world_cup_matches1_p
            where Home_Goals > Away_Goals),
            t3 as (
            select /*+ BROADCAST(t1) */ * from t1
            full join t2
            on t1.Away_Team = t2.Home_Team
            ),
            t4 as (
            select *,
            case when cnt_ta is null then 0 else cnt_ta end c1,
            case when cnt_th is null then 0 else cnt_th end c2
            from t3
            ),
            t5 as (
            select
            case when Away_Team is null then Home_Team
                 when Home_Team is null then Away_Team
                 else Away_Team
            end team,
            (c1+c2) cnt
            from t4
            )
            
            select * from t5
            
            ''').show(truncate=False)

+-------------+---+
|team         |cnt|
+-------------+---+
|Paraguay     |7  |
|Russia       |19 |
|Senegal      |3  |
|Sweden       |19 |
|Turkey       |5  |
|Germany      |67 |
|Ivory Coast  |3  |
|France       |34 |
|Greece       |2  |
|Algeria      |3  |
|Slovakia     |1  |
|Argentina    |43 |
|Wales        |1  |
|Belgium      |20 |
|Ecuador      |4  |
|Ghana        |4  |
|Peru         |5  |
|United States|8  |
|Chile        |11 |
|Croatia      |11 |
+-------------+---+
only showing top 20 rows

CPU times: user 1.97 ms, sys: 465 µs, total: 2.43 ms
Wall time: 819 ms


Задание 7. Для каждого чемпионата мира найдите количество забитых голов и количество сыгранных матчей для каждой команды. Используйте оконные функции для подсчета общего числа голов и матчей для каждой команды в рамках каждого чемпионата.


In [57]:
%%time
spark.sql('''             
            select *
            from ivanov.international_matches1_p
            
            ''').show(truncate=False)

+---+----------+----------+----------------+---------+----------+----------+-------------------+
|id |Tournament|Date      |Home_Team       |Away_Team|Home_Goals|Away_Goals|Home_Stadium_or_Not|
+---+----------+----------+----------------+---------+----------+----------+-------------------+
|1  |Friendly  |1872-11-30|Scotland        |England  |0         |0         |1                  |
|2  |Friendly  |1873-03-08|England         |Scotland |4         |2         |1                  |
|3  |Friendly  |1874-03-07|Scotland        |England  |2         |1         |1                  |
|4  |Friendly  |1875-03-06|England         |Scotland |2         |2         |1                  |
|5  |Friendly  |1876-03-04|Scotland        |England  |3         |0         |1                  |
|6  |Friendly  |1876-03-25|Scotland        |Wales    |4         |0         |1                  |
|7  |Friendly  |1877-03-03|England         |Scotland |1         |3         |1                  |
|8  |Friendly  |1877-03-05|Wal

In [58]:
%%time
spark.sql(''' 
            with t1 as (
            select Tournament, Home_Team, Away_Team, Home_Goals, Away_Goals,   
            row_number() over(partition by Tournament ORDER BY Tournament) rn
            from ivanov.international_matches1_p
            where Tournament != 'Friendly'),
            t2 as(
            select Tournament, Home_Team,
            sum(Home_Goals) over(partition by Home_Team ORDER BY Home_Team) sum_goal_h
            from t1
            ORDER BY Tournament
            ),
            t3 as (
            select DISTINCT(*),
            count(Home_Team) over(partition by Home_Team ORDER BY Home_Team) cnt_play_h
            from t2
            ORDER BY Tournament
            ),
            t4 as (
            select Tournament, Home_Team, Away_Team, Home_Goals, Away_Goals,   
            row_number() over(partition by Tournament ORDER BY Tournament) rn
            from ivanov.international_matches1_p
            where Tournament != 'Friendly'),
            t5 as(
            select Tournament, Away_Team,
            sum(Away_Goals) over(partition by Away_Team ORDER BY Away_Team) sum_goal_a
            from t4
            ORDER BY Tournament
            ),
            t6 as (
            select DISTINCT(*),
            count(Away_Team) over(partition by Away_Team ORDER BY Away_Team) cnt_play_a
            from t5
            ORDER BY Tournament),
            t7 as(
            select t3.Tournament, Home_Team as team, (sum_goal_h + sum_goal_a) as sum_goal,
            (cnt_play_h+cnt_play_a) cnt_play  from t3
            join t6 on t3.Tournament = t6.Tournament and t3.Home_Team = t6.Away_Team
            )
            
            select * from t7
            ''').show(truncate=False)

+-------------+--------------------+--------+--------+
|Tournament   |team                |sum_goal|cnt_play|
+-------------+--------------------+--------+--------+
|AFC Asian Cup|Iraq                |98      |99      |
|AFC Asian Cup|Cambodia            |6       |15      |
|AFC Asian Cup|Jordan              |31      |45      |
|AFC Asian Cup|Qatar               |551     |357     |
|AFC Asian Cup|India               |28      |45      |
|AFC Asian Cup|Kuwait              |132     |111     |
|AFC Asian Cup|Iran                |644     |300     |
|AFC Asian Cup|Thailand            |63      |99      |
|AFC Asian Cup|Hong Kong           |24      |59      |
|AFC Asian Cup|Israel              |52      |64      |
|AFC Asian Cup|Oman                |50      |83      |
|AFC Asian Cup|North Korea         |48      |63      |
|AFC Asian Cup|South Korea         |1111    |566     |
|AFC Asian Cup|Palestine           |20      |23      |
|AFC Asian Cup|Indonesia           |44      |68      |
|AFC Asian

In [59]:
df1 = spark.table('ivanov.international_matches1_p').cache()

In [60]:
%%time
spark.sql(''' 
            with t1 as (
            select Tournament, Home_Team, Away_Team, Home_Goals, Away_Goals,   
            row_number() over(partition by Tournament ORDER BY Tournament) rn
            from ivanov.international_matches1_p
            where Tournament != 'Friendly'),
            t2 as(
            select Tournament, Home_Team,
            sum(Home_Goals) over(partition by Home_Team ORDER BY Home_Team) sum_goal_h
            from t1
            ORDER BY Tournament
            ),
            t3 as (
            select DISTINCT(*),
            count(Home_Team) over(partition by Home_Team ORDER BY Home_Team) cnt_play_h
            from t2
            ORDER BY Tournament
            ),
            t4 as (
            select Tournament, Home_Team, Away_Team, Home_Goals, Away_Goals,   
            row_number() over(partition by Tournament ORDER BY Tournament) rn
            from ivanov.international_matches1_p
            where Tournament != 'Friendly'),
            t5 as(
            select Tournament, Away_Team,
            sum(Away_Goals) over(partition by Away_Team ORDER BY Away_Team) sum_goal_a
            from t4
            ORDER BY Tournament
            ),
            t6 as (
            select DISTINCT(*),
            count(Away_Team) over(partition by Away_Team ORDER BY Away_Team) cnt_play_a
            from t5
            ORDER BY Tournament),
            t7 as(
            select t3.Tournament, Home_Team as team, (sum_goal_h + sum_goal_a) as sum_goal,
            (cnt_play_h+cnt_play_a) cnt_play  from t3
            join t6 on t3.Tournament = t6.Tournament and t3.Home_Team = t6.Away_Team
            )
            
            select * from t7
            ''').show(truncate=False)

+-------------+--------------------+--------+--------+
|Tournament   |team                |sum_goal|cnt_play|
+-------------+--------------------+--------+--------+
|AFC Asian Cup|Iraq                |98      |99      |
|AFC Asian Cup|Cambodia            |6       |15      |
|AFC Asian Cup|Jordan              |31      |45      |
|AFC Asian Cup|Qatar               |551     |357     |
|AFC Asian Cup|India               |28      |45      |
|AFC Asian Cup|Kuwait              |132     |111     |
|AFC Asian Cup|Iran                |644     |300     |
|AFC Asian Cup|Thailand            |63      |99      |
|AFC Asian Cup|Hong Kong           |24      |59      |
|AFC Asian Cup|Israel              |52      |64      |
|AFC Asian Cup|Oman                |50      |83      |
|AFC Asian Cup|North Korea         |48      |63      |
|AFC Asian Cup|South Korea         |1111    |566     |
|AFC Asian Cup|Palestine           |20      |23      |
|AFC Asian Cup|Indonesia           |44      |68      |
|AFC Asian

In [61]:
df1.unpersist()

DataFrame[id: int, Tournament: string, Date: date, Home_Team: string, Away_Team: string, Home_Goals: int, Away_Goals: int, Home_Stadium_or_Not: int]

In [62]:
%%time
spark.sql(''' 
            with t1 as (
            select Tournament, Home_Team, Away_Team, Home_Goals, Away_Goals,   
            row_number() over(partition by Tournament ORDER BY Tournament) rn
            from ivanov.international_matches1_p
            where Tournament != 'Friendly'),
            t2 as(
            select Tournament, Home_Team,
            sum(Home_Goals) over(partition by Home_Team ORDER BY Home_Team) sum_goal_h
            from t1
            ORDER BY Tournament
            ),
            t3 as (
            select DISTINCT(*),
            count(Home_Team) over(partition by Home_Team ORDER BY Home_Team) cnt_play_h
            from t2
            ORDER BY Tournament
            ),
            t4 as (
            select Tournament, Home_Team, Away_Team, Home_Goals, Away_Goals,   
            row_number() over(partition by Tournament ORDER BY Tournament) rn
            from ivanov.international_matches1_p
            where Tournament != 'Friendly'),
            t5 as(
            select Tournament, Away_Team,
            sum(Away_Goals) over(partition by Away_Team ORDER BY Away_Team) sum_goal_a
            from t4
            ORDER BY Tournament
            ),
            t6 as (
            select DISTINCT(*),
            count(Away_Team) over(partition by Away_Team ORDER BY Away_Team) cnt_play_a
            from t5
            ORDER BY Tournament),
            t7 as(
            select t3.Tournament, Home_Team as team, (sum_goal_h + sum_goal_a) as sum_goal,
            (cnt_play_h+cnt_play_a) cnt_play  from t3
            join t6 on t3.Tournament = t6.Tournament and t3.Home_Team = t6.Away_Team
            )
            
            select /*+ BROADCAST(t6) */ * from t7
            ''').show(truncate=False)

+-------------+--------------------+--------+--------+
|Tournament   |team                |sum_goal|cnt_play|
+-------------+--------------------+--------+--------+
|AFC Asian Cup|Iraq                |98      |99      |
|AFC Asian Cup|Cambodia            |6       |15      |
|AFC Asian Cup|Jordan              |31      |45      |
|AFC Asian Cup|Qatar               |551     |357     |
|AFC Asian Cup|India               |28      |45      |
|AFC Asian Cup|Kuwait              |132     |111     |
|AFC Asian Cup|Iran                |644     |300     |
|AFC Asian Cup|Thailand            |63      |99      |
|AFC Asian Cup|Hong Kong           |24      |59      |
|AFC Asian Cup|Israel              |52      |64      |
|AFC Asian Cup|Oman                |50      |83      |
|AFC Asian Cup|North Korea         |48      |63      |
|AFC Asian Cup|South Korea         |1111    |566     |
|AFC Asian Cup|Palestine           |20      |23      |
|AFC Asian Cup|Indonesia           |44      |68      |
|AFC Asian

In [63]:
sc.stop()